# Computational prediction of drug-tager interactions

## Exploratory Data Analysis

Original article: Computational prediction of drug–target interactions using chemogenomic approaches: an empirical survey, A. Ezzat, others.
<br>Data link: http://web.kuicr.kyoto-u.ac.jp/supp/yoshi/drugtarget/

<p>

Data supplement. Organic molecules (Qm9 file): https://deepchemdata.s3-us-west-.amazonaws.com/datasets/molnet_publish/qm9.zip


## 1 - Pre-setup

### 1.1 - Imports (dependencies)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import os
import json
import requests
from tqdm import tqdm
import time



from io import StringIO #retrive information for mlflow
import sys #retrive information for mlflow


#Chemistry Libraries
from rdkit import Chem
from rdkit.Chem import AllChem



# Pubchem DB API https://pubchem.ncbi.nlm.nih.gov/compound/5388962
import pubchempy as pcp # to retrive features and SMILES

## 2 - Data imports

### 2.1 - Predicted drug-target interaction networks

Cinq types de données.
    <ul>Predicted compound-protein interacion pairs</ul>
    <ul>Binary relation list of the gold standard drug-target interaction data</ul>
    <ul>Adjacency matrix of the gold standard drug-target interaction data</ul>
    <ul>Compound structure similarity matrix</ul>
    <ul>Protein sequence similarity matrix</ul>

In [10]:
#relative paths. # Set directory paths for later use.
# Get the directory of the script file
base_dir = os.getcwd()
base_dir

ligants_type=['enzyme','GPCR','ion_channel','nuclear_receptor']

### 2.1.1 - GPCR

In [11]:
#GPCR

ltype=ligants_type[1]

#set tables
files_matrix_temp={'df_adjacency_matrix_GPCR_Y':'gpcr_admat_dgc.txt',
       'df_similarity_matrix_GPCR_compound_St':'gpcr_simmat_dc.txt',
       'df_similarity_matrix_GPCR_protein_Sd':'gpcr_simmat_dg.txt',
       }

df_temp_matrix={}


for df_name, file_name in files_matrix_temp.items():
    # Construct the file path using base_dir
    file_path = os.path.join(base_dir,'data','split',ltype, file_name)

    try:
        # Read the file
        print("Trying to read file at:", file_path) # Print the path for verification
        data_frame = pd.read_csv(file_path, delimiter='\t', index_col=0)
        df_temp_matrix[df_name] = data_frame
    except FileNotFoundError:
        print(f'File not found at the specified path: {file_path}')

df_adjacency_matrix_GPCR_Y=df_temp_matrix['df_adjacency_matrix_GPCR_Y']
df_similarity_matrix_GPCR_compound_St=df_temp_matrix['df_similarity_matrix_GPCR_compound_St']
df_similarity_matrix_GPCR_protein_Sd=df_temp_matrix['df_similarity_matrix_GPCR_protein_Sd']

Trying to read file at: C:\Users\riskf\OneDrive\DrugTargetSmilesBERT\data\split\GPCR\gpcr_admat_dgc.txt
Trying to read file at: C:\Users\riskf\OneDrive\DrugTargetSmilesBERT\data\split\GPCR\gpcr_simmat_dc.txt
Trying to read file at: C:\Users\riskf\OneDrive\DrugTargetSmilesBERT\data\split\GPCR\gpcr_simmat_dg.txt


In [12]:
#Adjacent matrix. Y
print('Lines (m): {}'.format(df_adjacency_matrix_GPCR_Y.shape[0]))
print('Columns (n): {}'.format(df_adjacency_matrix_GPCR_Y.shape[1]))
print('Size (m x n): {}'.format(df_adjacency_matrix_GPCR_Y.size))

number_interactions_enzimes=(df_adjacency_matrix_GPCR_Y.values == 1).sum()
print('Known interactions: {}'.format(number_interactions_enzimes))
print('Known interactions (%): {:.4f}%'.format(number_interactions_enzimes/df_adjacency_matrix_GPCR_Y.size*100))
print('No interactions: {}'.format(df_adjacency_matrix_GPCR_Y.size-number_interactions_enzimes))
print('No interactions(%): {:.4f}%'.format((df_adjacency_matrix_GPCR_Y.size-number_interactions_enzimes)/df_adjacency_matrix_GPCR_Y.size*100))


#print(df_adjacency_matrix_GPCR_Y.head(5))

Lines (m): 95
Columns (n): 223
Size (m x n): 21185
Known interactions: 635
Known interactions (%): 2.9974%
No interactions: 20550
No interactions(%): 97.0026%


In [13]:
#Similarity Matrix Compound Columns
print('Lines (m): {}'.format(df_similarity_matrix_GPCR_compound_St.shape[0]))
print('Columns (n): {}'.format(df_similarity_matrix_GPCR_compound_St.shape[1]))
print('Size (m x n): {}'.format(df_similarity_matrix_GPCR_compound_St.size))

#print(df_simmilarity_matrix_GPCR_compound_Sd.head(5))

Lines (m): 223
Columns (n): 223
Size (m x n): 49729


In [14]:
#Similarity Matrix Human Proteins Lines
print('Lines (m): {}'.format(df_similarity_matrix_GPCR_protein_Sd.shape[0]))
print('Columns (n): {}'.format(df_similarity_matrix_GPCR_protein_Sd.shape[1]))
print('Size (m x n): {}'.format(df_similarity_matrix_GPCR_protein_Sd.size))

#print(df_simmilarity_matrix_GPCR_protein_St.head(5))

Lines (m): 95
Columns (n): 95
Size (m x n): 9025


In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, diags
from numpy.linalg import inv

def calculate_objective(Y, A, B, W, Sd, St, lambda_l, lambda_d, lambda_t):
    # Convert sparse matrices to dense arrays
    Y_dense = Y.toarray() if hasattr(Y, "toarray") else Y
    W_dense = W.toarray() if hasattr(W, "toarray") else W

    #  error
    diff_matrix = Y_dense - A.dot(B.T)
    weighted_diff = W_dense * diff_matrix  # Element multiplication
    recon_error = np.linalg.norm(weighted_diff, 'fro')**2

    # Regularization terms
    reg_A = lambda_l * np.linalg.norm(A, 'fro')**2
    reg_B = lambda_l * np.linalg.norm(B, 'fro')**2

    # Similarity error 
    AAT = A.dot(A.T)
    Sd_dense = Sd.toarray() if hasattr(Sd, "toarray") else Sd
    sim_error_d = lambda_d * np.linalg.norm(Sd_dense - AAT, 'fro')**2

    BBT = B.dot(B.T)
    St_dense = St.toarray() if hasattr(St, "toarray") else St
    sim_error_t = lambda_t * np.linalg.norm(St_dense - BBT, 'fro')**2

    return recon_error + reg_A + reg_B + sim_error_d + sim_error_t

def update_A(Y, A, B, W, Sd, lambda_d, lambda_l, K):
    m, n = Y.shape
    A_new = np.zeros_like(A)
    for i in range(m):
        # Since Wi is a diagonal matrix formed from Wi, use Wi directly for weights
        weights_i = W[i].toarray().squeeze()  # Convert to dense format and get as 1D array
        
        # For Yi and Sdi
        Yi = Y[i].toarray().squeeze()
        Sdi = Sd[i].toarray().squeeze()

        # Initialize parts for summation
        part1_sum = np.zeros((K,))
        part2_sum = lambda_l * np.eye(K) + lambda_d * (A.T @ A)  # Regularization
        
        for j in range(n):
            # Use the weight for the i drug and j target directly
            weight_ij = weights_i[j]
            part1_sum += weight_ij * Yi[j] * B[j, :]
            part2_sum += weight_ij * np.outer(B[j, :], B[j, :])
        
        # Solve the linear system instead of direct inversion for numerical stability
        A_new[i, :] = np.linalg.solve(part2_sum, part1_sum)
    
    return A_new


def update_B(Y, A, B, W, St, lambda_t, lambda_l, K):
    m, n = Y.shape  # Assuming Y has dimensions m x n
    B_new = np.zeros_like(B)  # Initialize a new B matrix with the same shape as B
    for j in range(n):
        # Convert the j column of W to a dense format and extract as a 1D array
        weights_j = W[:, j].toarray().squeeze()
        
        # Convert the j column of Y to a dense format as a 1D array
        Yj = Y[:, j].toarray().squeeze()
        
        # Initialize the parts for summation
        part1_sum = np.zeros((K,))
        part2_sum = lambda_l * np.eye(K) + lambda_t * (B.T @ B)  # Regularizatio

        for i in range(m):  # Iterate over drugs
            # Use the extracted weight for the i drug and j target
            weight_ij = weights_j[i]
            part1_sum += weight_ij * Yj[i] * A[i, :]
            part2_sum += weight_ij * np.outer(A[i, :], A[i, :])

        # Solve the linear system instead of direct inversion for numerical stability
        B_new[j, :] = np.linalg.solve(part2_sum, part1_sum)

    return B_new

def generate_final_dataset(A, B, Y):
    final_dataset = []
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            features_drug = A[i, :]
            features_target = B[j, :]
            interaction_class = Y[i, j]
            final_dataset.append(np.concatenate([features_drug, features_target, [interaction_class]]))
    return np.array(final_dataset)

m, n = 95, 223  # Y matrix dimentions
K = 50  # feature dimension 
Y = csr_matrix(df_adjacency_matrix_GPCR_Y.values)  #  interaction matrix
St = csr_matrix(df_similarity_matrix_GPCR_compound_St.values)  #drug similarity matrix - compound
Sd = csr_matrix(df_similarity_matrix_GPCR_protein_Sd.values)  # target similarity matrix - protein
W = csr_matrix(np.ones((m, n)))  # Example weight matrix
A = np.random.rand(m, K)
B = np.random.rand(n, K)
count=0
start_time = time.time()

lambda_l, lambda_d, lambda_t = 0.1, 0.1, 0.01  # Regularization parameters
tolerance = 1e-3  # Convergence 
max_iterations = 1000  # Maximum number of iterations

objective_history = [calculate_objective(Y, A, B, W, Sd, St, lambda_l, lambda_d, lambda_t)]

for iteration in range(max_iterations):
    A = update_A(Y, A, B, W, Sd, lambda_d, lambda_l, K)
    B = update_B(Y, A, B, W, St, lambda_t, lambda_l, K)
    
    current_objective = calculate_objective(Y, A, B, W, Sd, St, lambda_l, lambda_d, lambda_t)
    objective_history.append(current_objective)
    
    if np.abs(objective_history[-1] - objective_history[-2]) < tolerance:
        print(f"Converged after {iteration} iterations.")
        elapsed_time = time.time() - start_time
        print(f"Interaction {count}/{max_iterations}. Time elapsed: {elapsed_time:.2f} seconds.", end='\r')    
        break
    else:
        #print(f"Interaction {count}/1000. Max of 1000 interactions")
        converg=objective_history[-1] - objective_history[-2]
        count +=1
        elapsed_time = time.time() - start_time
        remaining_interactions = max_iterations - count
        print(f"Interaction {count}/{max_iterations}. Remaining: {remaining_interactions}. Time elapsed: {elapsed_time:.2f} seconds. Convergence {converg}", end='\r')




Converged after 32 iterations.: 968. Time elapsed: 32.70 seconds. Convergence -0.0010272026124482636


In [16]:
#print(A)
#print(B)
#print(Y)

df_a=pd.DataFrame(A)
df_b=pd.DataFrame(B)
df_y=pd.DataFrame(Y)

file_path = os.path.join(base_dir,'data','split',ltype)
file_path

df_a.to_csv(file_path+'\df_a.csv', index=False)
df_b.to_csv(file_path+'\df_b.csv', index=False)
df_y.to_csv(file_path+'\df_y.csv', index=False)



In [17]:
##OUTPUT Matrix. Important for next steps.
final_dataset = generate_final_dataset(A, B, Y) 
final_df = pd.DataFrame(final_dataset)
file_name=f'final_new_par_{K}.csv'
file_path = os.path.join(base_dir,'data','split',ltype, file_name)
output_path = file_path

final_df.to_csv(output_path, index=False)
print("Final dataset saved!")

Final dataset saved!
